##3.4 斑点Blob
它使用拉普拉斯梯度，也就是一阶导极值点，二阶导零点。斑点对噪声很敏感，需要先做高斯平滑。先做高斯滤波，然后在做拉普拉斯滤波(也就是高斯滤波的二阶导)，和直接对高斯滤波做二次导是一样的

$\nabla^2[f(x,y) \otimes G(x,y) = \nabla^2G(x,y) \otimes f(x,y)]$

后面的叫做LoG

捕捉斑点，找拉普拉斯的极值，捕捉边缘，找拉普拉斯的临值